# Analysis of RF heating (high $q_x$)
This program is devoted to the analysis of RF heating simulations, done with MD program. Initially, first simulation were carried out in 2022, 17 of june (see `20220617`) but the aspect ratio of the potential was not close enough to 1. The problem was that the effect of the axial potential over radial direction was not properly considered. This implies to properly compute the beta parameter in a recursive way. The computation of the right beta is done in another Python program called `2021_Mathieu_parameters`. The data in `20220617` are viable, but do not have the symmetrical aspect ratio.
The high q data used in the RF heating article (2023) are ultimately analysed with this program ($q\geq 0.6$). Data is stored on Rivendel in the following folder `Rivendel/Simulations/20221006`. They are first pre-processed with a python program, `20220617_extractdatatonpz`, converting the raw data into `.npz` archives. Then the `.npz` archives are opened in this program and analysed.

This analysis program first opens the `.npz`, proposes to plot some temperature curves. A temporal window average is carried out, then the heating rate is numerically computed. Analytical cooling power is computed, along with heating rate from the spontaneous emission. Those computations are compared in a figure.

Keep in mind the analysis of low $q_x$ from $q_x=0.2$ to $q_x=0.6$ is done with another program called `20220930_Heating_rate_low_q-ForArticle`. Also, a previous program devoted to the analysis of those data and their fit with a sigmoid function was created. Its name is `20220617_exploitnpzdata-testfitinlin-ForArticle` and you should have a look at it.

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%pylab
from scipy.optimize import curve_fit

import tkinter as tk
from tkinter import filedialog

import os

from scipy import signal
import scipy.integrate as integrate
import matplotlib.ticker as ticker

# %matplotlib qt

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
%run functions
matplotlib.rcParams.update({'font.size': 25})
cm = pylab.get_cmap('tab10')
cm2 = pylab.get_cmap('Set1')
cm3 = pylab.get_cmap('turbo')

In [3]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.0e6*2*pi # 2.047e6
tauRF = 1/(Omega/2/pi)

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Open data stored as `.npz`
If you don't have the `.npz` archives please first convert the data using `20220617_extractdatatonpz` python program.

In [4]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

# Use 20220617_extractdatatonpz before

# Data is in the file
# /home/adrien/RemoteFS/Rivendel/Simulations/20221006/
# from Time_and_temp_RFHEAT_N0256_DC05_RF05.npz
# to Time_and_temp_RFHEAT_N0256_DC38_RF38.npz
# xxx in total

file_path = filedialog.askopenfilename(multiple=True) # initialdir = dir_string
# print(file_path)

time        = []
T_aux_avg   = []
r2_v2_rlim  = []
alpha       = []

for i,j in enumerate(file_path):
    print(j)
    with load(j) as data:
        time.append(data['time'])
        T_aux_avg.append(data['temp'])
        r2_v2_rlim.append(data['r2_v2_rlim'])
        alpha.append(data['alpha'])

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC11_RF11.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC12_RF12.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC13_RF13.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0512_DC03_RF03.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHE

In [5]:
def func_lin(t,a,b):
    return a*t+b
def func_exp(x,a,b):
    return b*exp(-a*x)
def func_pow(t,A,B):
    return t**A * 10**(B)
def func_pow2(t,A,B):
    return exp(B)*t**(A)

In [6]:
# Checking all the data directories
# You should have 49 files
# Rivendel/Simulations/20221006//Time_and_temp_RFHEAT_N0256_DC05_RF05.npz
# to
# Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N1024_DC38_RF38.npz
for i,j in enumerate(file_path):
    print(i,j)

0 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz
1 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
2 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
3 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
4 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz
5 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz
6 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC11_RF11.npz
7 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC12_RF12.npz
8 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC13_RF13.npz
9 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0512_DC03_RF03.npz
10 /home/adrien/RemoteFS/Rivendel/Simulations/2022

In [7]:
# Voltages and parameters used in the code

Urf = array([0, 0, 0, 0, 41.065, 55.35111, 57.40115, 59.45119, 61.50124, 63.55128, 65.60132, 67.65136, 69.70140, 71.75144, 0, 0, 0, 0, 0,
             64.6, 62.5, 62.83, 63.03, 63.24, 63.44, 63.86, 64.06, 64.27, 64.47, 62.7262, 62.93, 63.14, 63.34, 63.55, 63.75, 63.96, 64.16, 64.37, 65.74])
q = array([0, 0, 0, 0, 0.4, 0.54, 0.56,  0.58,  0.6, 0.62,  0.64,  0.66,  0.68,  0.7, 0, 0, 0, 0, 0,
           0.63, 0.61, 0.612, 0.614, 0.616, 0.618, 0.622, 0.624, 0.66, 0.68, 0.611, 0.613, 0.615, 0.617, 0.619, 0.621, 0.623, 0.625, 0.627, 0.629])
Udc = array([0, 0, 0, 0, 5.706, 10.41491735, 11.26284989, 12.15184042, 13.08323245, 14.05846827, 15.07909652, 16.14678051, 17.2633073, 18.43059758, 0, 0, 0, 0, 0,
             16.0, 14.8, 14.95, 15.07, 15.19, 15.31, 15.56, 15.68, 15.81, 15.94, 14.89, 15.01, 15.13, 15.25, 15.37, 15.50, 15.62, 15.75, 15.87, 16.00])
N_ions = array([256,256,256,256,256,256,256,256,256,512,512,512,512,512,512,512,512,512,512,512,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,
                1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024])
# K = 1e5*2*pi
# beta_guess = 0.423
# a = 2*K**2/(Omega**2)*Udc
# for k in range(50):
#     beta_guess = beta_continue_alamano(a,q,beta_guess)
#     beta_guess = sqrt(beta_guess)

# omega_z_2 = ( K )**2 * array(Udc)
# omega_x_2 = ( beta_guess*Omega/2 )**2
# omega_r_2 = omega_x_2 - 0.5*omega_z_2

K = 1e5*2*pi
print('q =',q)
beta_guess = 0.423
omega_z_2 = K**2*Udc
az = 2*K**2/Omega**2*Udc
au = -0.5 * az
for i in range(50):
    for k in range(50):
        beta_guess = beta_continue_alamano(au,q,beta_guess)
        beta_guess = sqrt(beta_guess)
    omega_r_2 = ( beta_guess*Omega/2 )**2
    au = -2*omega_r_2/Omega**2

# display for one given run
to_fit = 2
my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
print(sqrt(omega_r_2[9])/2/pi)
print(sqrt(omega_z_2[9])/2/pi)

x  = time[to_fit]-1e-3
fs = 1/diff(x)[1]

q = [0.    0.    0.    0.    0.4   0.54  0.56  0.58  0.6   0.62  0.64  0.66
 0.68  0.7   0.    0.    0.    0.    0.    0.63  0.61  0.612 0.614 0.616
 0.618 0.622 0.624 0.66  0.68  0.611 0.613 0.615 0.617 0.619 0.621 0.623
 0.625 0.627 0.629]
374946.23973188916
374946.2397464469


In [8]:
# Display parameters of simus
# in nice table

print('|   cond  | N  | q_x |   f_r   |   f_z   |   ~a_x  | R/L | wz2/wr2 | beta|',end='\n')
print('|---------|----|-----|---------|---------|---------|-----|---------|-----|',end='\n')

for cond in range(len(file_path)):
    #print(r2_v2_rlim[myRF][k+6][50]*1000) # [condition][dim][time]
    my_try = int(file_path[cond][file_path[cond].find('DC')+2:file_path[cond].find('DC')+4])
    
    print(f'|DC{my_try:02d}_RF{my_try:02d}|{N_ions[cond]:04d}|{q[my_try]:.3f}|{sqrt(omega_r_2)[my_try]/2/pi:.3e}|{sqrt(omega_z_2)[my_try]/2/pi:.3e}|{omega_z_2[my_try]/Omega**2*2:.3e}|{alpha[cond]:.3f}|{omega_z_2[my_try]/omega_r_2[my_try]:09.06f}|{beta_guess[my_try]:.3f}|',end='\n')

|   cond  | N  | q_x |   f_r   |   f_z   |   ~a_x  | R/L | wz2/wr2 | beta|
|---------|----|-----|---------|---------|---------|-----|---------|-----|
|DC05_RF05|0256|0.540|3.227e+05|3.227e+05|5.207e-02|1.099|01.000000|0.323|
|DC06_RF06|0256|0.560|3.356e+05|3.356e+05|5.631e-02|1.125|01.000000|0.336|
|DC07_RF07|0256|0.580|3.486e+05|3.486e+05|6.076e-02|1.100|01.000000|0.349|
|DC08_RF08|0256|0.600|3.617e+05|3.617e+05|6.542e-02|1.120|01.000000|0.362|
|DC09_RF09|0256|0.620|3.749e+05|3.749e+05|7.029e-02|1.126|01.000000|0.375|
|DC10_RF10|0256|0.640|3.883e+05|3.883e+05|7.540e-02|1.143|01.000000|0.388|
|DC11_RF11|0256|0.660|4.018e+05|4.018e+05|8.073e-02|1.161|01.000000|0.402|
|DC12_RF12|0256|0.680|4.155e+05|4.155e+05|8.632e-02|1.169|01.000000|0.415|
|DC13_RF13|0256|0.700|4.293e+05|4.293e+05|9.215e-02|1.176|01.000000|0.429|
|DC03_RF03|0512|0.000|0.000e+00|0.000e+00|0.000e+00|1.095|000000nan|0.000|
|DC04_RF04|0512|0.400|2.352e+05|2.389e+05|2.853e-02|1.077|01.031086|0.235|
|DC05_RF05|0512|0.540|3.2

/tmp/ipykernel_191130/1106110854.py:11: RuntimeWarning: invalid value encountered in double_scalars
  print(f'|DC{my_try:02d}_RF{my_try:02d}|{N_ions[cond]:04d}|{q[my_try]:.3f}|{sqrt(omega_r_2)[my_try]/2/pi:.3e}|{sqrt(omega_z_2)[my_try]/2/pi:.3e}|{omega_z_2[my_try]/Omega**2*2:.3e}|{alpha[cond]:.3f}|{omega_z_2[my_try]/omega_r_2[my_try]:09.06f}|{beta_guess[my_try]:.3f}|',end='\n')


In [234]:
# write those parameters in a file
with open('data_parameters.txt','w') as f:
    f.write('2022 oct 20\n')
    f.write('Paramètres des simulations de chauffage RF\n')
    f.write('Simulations dans le dossier Rivendel/Simulations/20220617\n\n')
    
    f.write('|   cond  | N  | q_x|   f_x   |   f_z   |   ~a_x  | R/L | beta|\n')
    f.write('|---------|----|----|---------|---------|---------|-----|-----|\n')

    for cond in range(len(file_path)):
        #print(r2_v2_rlim[myRF][k+6][50]*1000) # [condition][dim][time]
        my_try = int(file_path[cond][file_path[cond].find('DC')+2:file_path[cond].find('DC')+4])

        f.write(f'|DC{my_try:02d}_RF{my_try:02d}|{N_ions[cond]:04d}|{q[my_try]:.2f}|{sqrt(omega_x_2)[my_try]/2/pi:.3e}|{sqrt(omega_z_2)[my_try]/2/pi:.3e}|{omega_z_2[my_try]/Omega**2*2:.3e}|{alpha[cond]:.3f}|{beta_guess[my_try]:.3f}|\n')

# Some plots of temperature

In [38]:
# plot temperature with fit

start_for_fit = 200
T_crit = 0.5
print('file to analyze')

to_fit = 21+2 #11 19 8
temp_to_end = 100

my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
print('>',to_fit,file_path[to_fit])
x  = time[to_fit]+1e-3
y  = T_aux_avg[to_fit]


figname = 'T_vs_time'
figure(figname,clear='True')
#xlim(0.8,6)
ax1 = subplot(111)
ax1.grid()
ax1.semilogy(x[::20]*1e3-2, y[::20] ,label=r'$T$',color='C0',lw=4)
# ax1.semilogy(x*1e3, y_filt ,label=r'$T$ filtered',color='C1',ls=':',lw=3)
# ax2.vlines(x[end_fit]*1e3,1e-2,150,color='k')

nticks = 9
maj_loc = ticker.LogLocator(numticks=nticks)
min_loc = ticker.LogLocator(subs='all', numticks=nticks)
ax1.yaxis.set_major_locator(maj_loc)
ax1.yaxis.set_minor_locator(min_loc)

ax1.set_xlabel('t (ms)',fontsize=35)
ax1.set_ylabel('T (K)',fontsize=35)
ax1.tick_params(axis='both', labelsize=35)
ax1.set_xlim(-2.15,18)
ax1.set_ylim(8e-4,1100)
tight_layout()
subplots_adjust(top=0.90,
bottom=0.25)
savefig(figname+'1024.eps',dpi=300)

file to analyze
> 23 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N1024_DC08_RF08.npz


In [10]:
to_fit = 1+2 #11 19 8
temp_to_end = 100
my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
print('>',to_fit,file_path[to_fit])
x  = time[to_fit]-1e-3
y  = T_aux_avg[to_fit]
ax1.semilogy(x[::20]*1e3, y[::20] ,label=r'$T$',color='C3',lw=1)
# ax1.semilogy(x*1e3, y_filt ,label=r'$T$ filtered',color='C1',ls=':',lw=3)

# ax1.text(6,12, '$N=1024$', fontsize=30, ha='right')
# ax1.text(27,12, '$N=256$', fontsize=30, ha='right')

ax1.set_xlabel('t (ms)')
ax1.set_ylabel('T (K)')
ax1.set_xlim(-1,37)
ax1.set_ylim(8e-4,510)
tight_layout()
subplots_adjust(top=0.96,
bottom=0.18)

# ax2 = subplot(212)
# ax2.grid()
# ax2.plot(x*1e3, y ,label=r'$T$',color='C0',lw=0.3)
# ax2.plot(x*1e3, y_filt ,label=r'$T$ filtered',color='C1',ls=':',lw=3)
# y_fit5 = func5(x[start_for_fit:end_fit],*popt)
# ax2.plot(x[start_for_fit:end_fit]*1e3,10**y_fit5,color='r',ls='--',label=r'$\psi$',lw=2)
# ax2.set_ylim(-10,175)
# ax2.vlines(x[end_fit]*1e3,1e-2,150,color='k')


# savefig(figname+'.eps',dpi=300)

> 3 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz


# Window average
Data are averaged of a temporal window including `window_samp` samples. `x_win` is the new time array. `y_win` is the averaged temperature corresponding to the `x_win`. `diff(y_win[to_plot])/window_samp` is the numerically computed heating rate $H$.

In [9]:
# Windowing temperature
window_samp = 200
x_win = [] # zeros((len(y)//window_samp,len(file_path)))
y_win = [] # zeros((len(y)//window_samp,len(file_path)))

for to_fit in range(len(file_path)):
#     print(to_fit)
    x_win.append([])
    y_win.append([])
    x = time[to_fit]-1e-3
    y = T_aux_avg[to_fit]
    for i in range(len(y)//window_samp-1):
        x_win[to_fit].append( (x[i*window_samp]+x[(i+1)*window_samp])/2 )
        y_win[to_fit].append( sum(y[i*window_samp:(i+1)*window_samp])/window_samp )
    
print(diff(x_win[0])[0]*1e3,tauRF*window_samp*1e3)

0.10000000000000005 0.09999999999999999


In [10]:
# Plot windowed temperature
to_plot = 25
x  = time[to_plot]-1e-3
y  = T_aux_avg[to_plot]
figure('Windowed T',clear='True')
semilogy(x*1e3, y,':')
semilogy(array(x_win[to_plot])*1e3, y_win[to_plot],ls='',marker='+',ms=10,mew=2)
xlabel('t [ms]')
ylabel('T [K]')
# hlines(500,0,0.02)
grid()
tight_layout()

In [11]:
figname = 'Windowed_dT_dt'
fig = figure(figname,clear='True')
fig.set_size_inches(11.7, 8.3)
ax = fig.add_subplot(111)
# ax.semilogy(x_win[:-1]*1e3, diff(y_win)/window_samp,'+')
for to_plot in [0,1,2,3,4,5,6]: # [21,23,25,27] [12,14,16,18] [1,3,5,7]
    my_try = int(file_path[to_plot][file_path[to_plot].find('DC')+2:file_path[to_plot].find('DC')+4])
    print(file_path[to_plot])
    ax.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
              marker='.',ms=20,mec='k',mew=0.3,ls='',
              label=f'{q[my_try]}')
# semilogy(x[::100]*1e3, y[::100],':',color='y')
# ax.set_xlabel('t [ms]')
ax.set_xlabel('T [K]')
ax.set_ylabel('$H=dT/dt$ [K/RF period]')
ax.set_xlim(3e-3,11)
ax.grid()
legend(title='$q_x$, $N=1024$',loc=0)
tight_layout()
# savefig(figname+'N1024_zoom'+'.eps',dpi=300)

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC11_RF11.npz


In [12]:
# Plot windowed temperature
# figure('Windowed T',clear='True')
# semilogy(x_win*1e3, y_win,'+')
# semilogy(x[::100]*1e3, y[::100],':',color='y')
# grid()
# tight_layout()
figname = 'Windowed_dT_dt'
fig = figure(figname,clear='True')
fig.set_size_inches(11.7, 8.3)
ax = fig.add_subplot(111)
# ax.semilogy(x_win[:-1]*1e3, diff(y_win)/window_samp,'+')
incr = 0
for to_plot in [3,14,23]: # [4,15,24]
    print(file_path[to_plot])
    my_try = int(file_path[to_plot][file_path[to_plot].find('DC')+2:file_path[to_plot].find('DC')+4])
    ax.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
              marker='.',ms=20,mec='k',mew=0.3,ls='',
              label=f'{N_rework[incr]}')
    incr+=1
# semilogy(x[::100]*1e3, y[::100],':',color='y')
# ax.set_xlabel('t [ms]')
ax.set_xlabel('T [K]')
ax.set_ylabel('$H=\mathrm{d}T/\mathrm{d}t$ [K/RF period]')
# ax.set_xlim(3e-3,11)
ax.grid()
legend(title=f'$N$, $q_x={q[my_try]}$',loc=4)
tight_layout()

# savefig(figname+'q0.60'+'.eps',dpi=300)

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz


NameError: name 'N_rework' is not defined

# Computation of heating and cooling powers
Spontaneous emission heating power rate $H_e$ and cooling power rate $G$ will be computed in the next cells, then compared to the numerically computed heating power rate $H$.

# Functions used to describe ions
### From fluo-variations_optimal-temp.ipynb
I present the functions used to compute fluorescence and cooling power as a function of the temperature. See Foot chapter 7 and 9. The two basic functions are as follows.

- $\texttt{MB}(v)$ is the Maxwell-Boltzmann distribution for a given temperature. It provides the probability to find an atom with a given velocity in a gas with temperature T.

- $\texttt{pfl_dop}(v) = \rho_{ee}(v)$ is the atomic ray profile with Doppler effect. It provides the probability of excitation of a single oscillator given Rabi frequency, detuning, lambda and its velocity. It is considered equal to the excited population, i.e the proportion of excited atoms. This function is used with fixed Rabi frequency, detuning and lambda.

Those functions are used to compute the total fluorescence of the ion ensemble under laser cooling $F$, the cooling power of laser $G$, the heating induced by the spontaneous emission $G_{Hot}$.

- $\texttt{prob_fluo} = \texttt{pfl_dop}\times \texttt{MB}$ is the probability of excitation of an ion with velocity v in an ensemble at temperature $T$. The sum $F=\Gamma\sum_v \texttt{prob_fluo}\; dv$ is the fluorescence emitted by an ensemble of ions with given temperature $T$.

- $\texttt{cool_power} = \texttt{pfl_dop}\times \texttt{MB}\times kv$ is used to compute the cooling energy of laser $G = \sum_v \texttt{cool_power}\; \mathrm{d}v \times \;\hbar k \Gamma$ [J].

- $\texttt{hot_power} = \texttt{prob_fluo}$ is used to compute the heating induced by the spontaneous emission $G_{Hot} = \sum_v \texttt{hot_power}\; \mathrm{d}v \times \;\hbar^2 k^2 \Gamma/m$ [J].

You can multiply $G$ and $G_{Hot}$ by $1/(k_b\tau_{RF})$ to express those quantities in K/RF period. This transformation relies on the following formula : E = 3/2 k_bT.


$\texttt{MB} = \sqrt{\frac{m}{2\pi k_BT}}\exp{-\frac{mv^2}{2k_BT}}$

$\texttt{pfl_dop} = \rho_{ee} = \frac{0.25\Omega_R^2}{0.5\Omega_R^2 + 0.25\Gamma^2 + \Delta^2} = \frac{A/2}{A+B + \Delta^2}$

$\texttt{cool_power} = \texttt{pfl_dop}\times \texttt{MB}\times kv$

In [13]:
# Doppler profile (Probability)
def pfl_dop(v, delta, k, Rab, Gam):
    return .25*Rab**2/(0.5*Rab**2+.25*Gam**2+(delta-k*v)**2)

# Maxwell-Boltzmann distribution
def MB(v, T):
    kb = 1.38064852*1e-23  # Boltzman
    m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
    return (m_Ca/(np.pi*2*kb*T))**(1/2) * np.exp(-m_Ca*v**2/(2*kb*T)) # **1 car vitesse par rapport au laser compte seulement (1D)

# Probabilities product
# For a range of velocities,
# compute the prob_fluo
# for a given T, delta, Rab, Gam
def prob_fluo(vmin, vmax, nv, T, delta, k, Rab, Gam):
    nu = k*np.linspace(vmin, vmax, nv)
    return nu, pfl_dop(-nu/k, delta, k, Rab, Gam)*MB(nu/k, T)

# unit is of kv as Fscatt/Gamma
def cool_power(vmin, vmax, nv, T, delta, k, Rab, Gam):
    nu = k*np.linspace(vmin, vmax, nv)
    return nu, pfl_dop(-nu/k, delta, k, Rab, Gam)*MB(nu/k, T)*nu

def hot_power(vmin, vmax, nv, T, delta, k, Rab, Gam):
    nu = k*np.linspace(vmin, vmax, nv)
    return nu, pfl_dop(-nu/k, delta, k, Rab, Gam)*MB(nu/k, T)

def T_lim(delta):
    return -0.5*hbar*Gam**2*(1+(2*delta/Gam)**2) / (4*delta)/kb

In [14]:
# Laser parameters
# For cooling
lam = 397e-9 # m 396.84620*1e-9
klam = 2*pi/lam # m^-1
Gam = 21570000.0 *2*pi
delta = Gam
I = 170 # W/m²
Rab = Gam

c_light = 299792458
h_pl = 6.62607015*1e-34
h_pl_bar = h_pl/2/pi
gamma_SP = 21.57*1e6
tau = 1/gamma_SP
tau_RF = 1/(Omega/(2*pi))

# Check that the integral of MB dist is 1
T = 0.01
v_thermique = sqrt(2*kb*T/m_Ca)
nv = 2000
dv = 6*v_thermique/nv
dist_MB = []
for v in linspace(-3*v_thermique,3*v_thermique,nv):
    dist_MB.append(MB(v, T))
print(sum(dist_MB)*dv)

0.9994781288008786


In [15]:
# Computation heating power H, spontaneous emission heating power H_e and cooling power

nv = 1000
log_temps = linspace(-3.5,3,nv)
coolpow_05 = []
coolpow_1 = []
# coolpow_1_bis = []
coolpow_2 = []
# coolpow_4 = []
# coolpow_6 = []
# coolpow_8 = []
# coolpow_10 = []
# coolpow_50 = []
hotpow   = []
hotpow_1 = []
for T in 10**log_temps:
    v_thermique = sqrt(2*kb*T/m_Ca)
    dv = 10*v_thermique/nv
    coolpow_05.append( sum(cool_power(-dv*nv/2, dv*nv/2, nv, T, -Gam/2, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF ) # sum()*Gamma*dv = Fv
    coolpow_1.append( sum(cool_power(-dv*nv/2, dv*nv/2, nv, T, -Gam, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_1_bis.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam, k, 2*Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_2.append( sum(cool_power(-dv*nv/2, dv*nv/2, nv, T, -2*Gam, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_4.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -4*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_6.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -6*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_8.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -8*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_10.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -10*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_50.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -50*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    hotpow.append( sum(hot_power(-dv*nv/2, dv*nv/2, nv, T, -Gam/2, klam, Rab, Gam))*dv*h_pl_bar**2*klam**2/m_Ca*Gam/kb*tau_RF )
    hotpow_1.append( sum(hot_power(-dv*nv/2, dv*nv/2, nv, T, -Gam, klam, Rab, Gam))*dv*h_pl_bar**2*klam**2/m_Ca*Gam/kb*tau_RF )

In [25]:
# Plot windowed temperature
# figure('Windowed T',clear='True')
# semilogy(x_win*1e3, y_win,'+')
# semilogy(x[::100]*1e3, y[::100],':',color='y')
# grid()
# tight_layout
my_color=[7,2,6,3] # 
figname = 'dT_vs_T_g_3N'
fig = figure(figname,clear='True')
fig.set_size_inches(11.7, 6) # 8.3
ax1 = fig.add_subplot(111)

# ax.semilogy(x_win[:-1]*1e3, diff(y_win)/window_samp,'+')
incr = 0
choosen_plot = [3,14,23]
my_marker=['v','D','P','.']
my_size = [17,13,17,17]
for to_plot in choosen_plot: # [4,15,24] 1,12,21 3,14,23
    print(file_path[to_plot])
    my_try = int(file_path[to_plot][file_path[to_plot].find('DC')+2:file_path[to_plot].find('DC')+4])
    ax1.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
              marker=my_marker[incr],ms=my_size[incr],mec='k',mew=0.2,ls='',color=cm(incr/8) )#,
              #label=f'{N_rework[incr]}')
    incr+=1

# to_plot = 20
# ax1.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
#           marker='.',ms=14,mec='k',mew=0,ls='',color=cm(4/8),
#           label=f'{N_rework[2]}')

ax1.loglog(y_win[choosen_plot[0]][275:-100],
           diff(y_win[choosen_plot[0]][275:-99])/window_samp,
          marker='',ls=':',lw=2,color=cm(0/8))
ax1.loglog(y_win[choosen_plot[1]][120:-135],
           diff(y_win[choosen_plot[1]][120:-134])/window_samp,
          marker='',ls=':',lw=2,color=cm(1/8))
ax1.loglog(y_win[choosen_plot[2]][60:-90],
           diff(y_win[choosen_plot[2]][60:-89])/window_samp,
          marker='',ls=':',lw=2,color=cm(2/8))

# ax1.loglog(y_f,y_fit = func5(x,*popt_smooth[to_fit,:]))

howdoyouturnthison = False
if howdoyouturnthison == True:
#     ax1.loglog(10**log_temps,coolpow_05,ls="-",color='xkcd:red',lw=4,label='$-0.5\Gamma$')
    ax1.loglog(10**log_temps,coolpow_1,ls="-",color='xkcd:red',lw=4,label='$-\Gamma$')
    ax1.loglog(10**log_temps[:nv-300],hotpow_1[:nv-300],ls='-',color='k',lw=4,label='$H_e$') # 'dashdot'
else:
#     ax1.loglog(10**log_temps,coolpow_05,ls="-",color='xkcd:red',lw=4)
    ax1.loglog(10**log_temps,coolpow_1,ls="-",color='xkcd:red',lw=4)
    ax1.loglog(10**log_temps[:nv-300],hotpow_1[:nv-300],ls='-',color='k',lw=4) # 'dashdot'

ax1.set_xlabel('$T$ (K)')
ax1.set_ylabel(r'$dT/dt$ (units of K/$\tau_{\mathrm{rf}}$)')
ax1.grid()
ax1.grid(True, which="minor", ls=":", color='0.80')
ax1.set_xlim(3e-3,555)
ax1.set_ylim(1e-9,0.4)
# ax1.legend(title=f'$N$',loc=4,ncol=1,fontsize=22)

# old part for showing q dependency
# replaced by 20220930_Heating_rate_low_q-ForArticle

# ax2 = fig.add_subplot(122,sharey=ax1,sharex=ax1)
# # ax.semilogy(x_win[:-1]*1e3, diff(y_win)/window_samp,'+')
# incr = 0
# # ax2.loglog(y_win[49][:-1], diff(y_win[49])/window_samp,
# #               marker='.',ms=17,mec='k',mew=0.2,ls='',color=cm((5+1)/8),
# #               label=f'0.4')
# # ax2.loglog(y_win[49][len(y_win[49])//2+325:len(y_win[49])//2+350],
# #            diff(y_win[49][len(y_win[49])//2+325:len(y_win[49])//2+350+1])/window_samp,
# #           marker='',ls=':',color=cm(6/8))
# for to_plot in [21,23,25,27]: # [21,23,25,27] [12,14,16,18] [1,3,5,7]
#     my_try = int(file_path[to_plot][file_path[to_plot].find('DC')+2:file_path[to_plot].find('DC')+4])
#     print(file_path[to_plot])
#     ax2.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
#               marker='.',ms=17,mec='k',mew=0.2,ls='',
#               label=f'{q[my_try]}',color=cm2(my_color[incr]/8))
#     incr+=1
    
# ax2.loglog(y_win[21][len(y_win[21])//2+100:len(y_win[21])//2+120],
#            diff(y_win[21][len(y_win[21])//2+100:len(y_win[21])//2+120+1])/window_samp,
#           marker='',ls=':',color=cm2(7/8))
# ax2.loglog(y_win[23][len(y_win[23])//2-30:len(y_win[23])//2-12],
#            diff(y_win[23][len(y_win[23])//2-30:len(y_win[23])//2-12+1])/window_samp,
#           marker='',ls=':',color=cm2(2/8))
# ax2.loglog(y_win[25][len(y_win[25])//2-20:len(y_win[25])//2+1],
#            diff(y_win[25][len(y_win[25])//2-20:len(y_win[25])//2+1+1])/window_samp,
#           marker='',ls=':',color=cm2(6/8))
# ax2.loglog(y_win[27][len(y_win[27])//2-45:len(y_win[27])//2-15],
#            diff(y_win[27][len(y_win[27])//2-45:len(y_win[27])//2-15+1])/window_samp,
#           marker='',ls=':',color=cm2(3/8))

# ax2.loglog(10**log_temps,coolpow_05,ls="-",lw=2,color='k',label='-$0.5\Gamma$')
# ax2.loglog(10**log_temps,coolpow_1,ls="--",lw=2,color='k',label='-$\Gamma$')
# ax2.loglog(10**log_temps,coolpow_2,ls=":",lw=2.5,color='k',label='$-2\Gamma$')  
# ax2.loglog(10**log_temps[:nv-300],hotpow[:nv-300],linestyle='-',color='xkcd:red',lw=2.5,label='$H_e$') 
# # semilogy(x[::100]*1e3, y[::100],':',color='y')
# # ax.set_xlabel('t [ms]')
# ax2.set_xlabel('T (K)')
# ax2.grid(True, which="minor", ls=":", color='0.80')
# ax2.set_ylim(1e-9,1)
# ax2.set_xlim(3e-3,500)
# ax2.grid()
# ax2.set_yticks([1e-9,1e-8,1e-7,1e-6,1e-5,1e-4,1e-3,1e-2,1e-1,1])
# ax2.legend(title='$q_x$, $N=1024$',loc=4,ncol=2,fontsize=18)

tight_layout()

savefig(figname+'.eps',dpi=300)
# savefig(figname+'varq_varN_with0.60_guides'+'.eps',dpi=300)
# savefig(figname+'varq_varN_with0.60_guides_0.4bonus'+'.jpg',dpi=300)
# savefig(figname+'varq_varN_with0.60_guides'+'.png',dpi=300)

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0512_DC08_RF08.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N1024_DC08_RF08.npz


In [20]:
# Plot windowed temperature
# figure('Windowed T',clear='True')
# semilogy(x_win*1e3, y_win,'+')
# semilogy(x[::100]*1e3, y[::100],':',color='y')
# grid()
# tight_layout

my_color=[7,2,6,3] # 
figname = 'Windowed_dT_dt_q_N_zoomq'
fig = figure(figname,clear='True')
fig.set_size_inches(11.7, 8.3)
ax1 = fig.add_subplot(111)

# ax.semilogy(x_win[:-1]*1e3, diff(y_win)/window_samp,'+')
incr = 0
choosen_plot = [1,30,31,40,36]
for to_plot in choosen_plot: # [4,15,24] 1,12,21 3,14,23
    print(file_path[to_plot])
    my_try = int(file_path[to_plot][file_path[to_plot].find('DC')+2:file_path[to_plot].find('DC')+4])
    ax1.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
              marker='.',ms=17,mec='k',mew=0.2,ls=':',color=cm(incr/8),
              label=f'{q[my_try]:.3f}')
    incr+=1

# ax1.loglog(y_f,y_fit = func5(x,*popt_smooth[to_fit,:]))

# ax1.loglog(0,0,marker='',ls='',label=' ')
    
ax1.loglog(10**log_temps,coolpow_05,ls="-",color='k',lw=2,label='$-0.5\Gamma$')
ax1.loglog(10**log_temps,coolpow_1,ls="--",color='k',lw=2,label='$-\Gamma$')
# ax1.loglog(10**log_temps,coolpow_1_bis,ls="--",color='r',label='$-\Gamma$')
ax1.loglog(10**log_temps,coolpow_2,ls=":",color='k',lw=2.5,label='$-2\Gamma$')
ax1.loglog(10**log_temps[:nv-300],hotpow[:nv-300],ls='-',color='xkcd:red',lw=2.5,label='$H_e$') # 'dashdot'


ax1.set_xlabel('T (K)')
ax1.set_ylabel('$\mathrm{d}T/\mathrm{d}t$ (K/RF period)')
ax1.grid()
ax1.grid(True, which="minor", ls=":", color='0.80')
ax1.legend(title=f'$N=1024$, $q_x$',loc=4,ncol=2,fontsize=18)

tight_layout()

# savefig(figname+'special_q'+'.eps',dpi=300)
# savefig(figname+'varq_varN_with0.60_guides_0.4bonus'+'.jpg',dpi=300)
# savefig(figname+'special_q'+'.png',dpi=300)

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N1024_DC20_RF20.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N1024_DC21_RF21.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N1024_DC30_RF30.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N1024_DC26_RF26.npz
